In [1]:
push!(LOAD_PATH, "../src")
using VariableHeightInvertedPendulum
using PyCall
using PyPlot
using Polynomials
using ForwardDiff
# using ODE

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
test = false
g = 9.8
x0 = -0.3
z0 = 1.
zd0 = 0.
zf = 1.
orbital_energies = test ? Dict("Eo_negative" => 0.) : Dict("Eo_negative" => -0.1, "Eo_zero" => 0., "Eo_positive" => 0.1);

In [3]:
function run_scenario(scenario::Scenario; kwargs...)
    filename = "$(scenario.model.name)_$(scenario.name).mp4"
    sim_movie(scenario.model, scenario.initial_conditions, filename; test = test, kwargs...)
end

run_scenario (generic function with 1 method)

In [4]:
models = []
push!(models, LIPM("lipm", g, zf))
push!(models, PolynomialVariableHeightModel("slope", g, Poly([zf, 0.2])))
push!(models, PolynomialVariableHeightModel("variable_height", g, Poly([zf, 0.2, 0.5])));

scenarios = Scenario[]
for model in models
    for kv in orbital_energies
        scenario_name = first(kv)
        Eo = last(kv)
        xd0 = velocity_given_orbital_energy(model, x0, Eo)
        initial_conditions = [x0; xd0]
        scenario = Scenario(scenario_name, model, initial_conditions)
        push!(scenarios, scenario)
    end
end

In [5]:
for scenario in scenarios
    run_scenario(scenario)
end

In [6]:
function run_solved_trajectory(name, xd0; kwargs...)
    traj = zero_orbital_energy_trajectory(x0, z0, xd0, zd0, g, zf)
    trajectory_solved_model = PolynomialVariableHeightModel("trajectory_solved", g, traj)
    run_scenario(Scenario(name, trajectory_solved_model, [x0; xd0]); restrict_ztraj = true, kwargs...)
end

run_solved_trajectory (generic function with 1 method)

In [7]:
run_solved_trajectory("fast", 1.)
run_solved_trajectory("slow", 0.9)
run_solved_trajectory("too_slow", 0.8)

In [8]:
run_solved_trajectory("fast_w_region", 1.; show_region = true)
run_solved_trajectory("too_slow_w_region", 0.8; show_region = true)